# System Public API Examples

This notebook provides basic examples of how to use System's public API. Full documentation is available on our [API documentation site](https://systeminternal.github.io/api/).

Running these examples requires you to have an API key, provided by System. Please contact us at [api@system.com](mailto:api@system.com) if you are interested in exploring our public API.

## 1. Import libraries and define helper functions

In [ ]:
!pip install api-client==1.28.0 --extra-index-url https://pypi.fury.io/systeminc/

In [3]:
import api_client
import json
import os

In [4]:
def pprint_topic(t):
    print(f"{t.name} (ID: {t.id}, Wikidata ID: {t.wikidata_id})")
    
def pprint_determinant(d):
    print(f"{d.topic.name} (ID: {d.id}, Number of findings: {d.num_findings}, Median effect size: {d.median_effect_size})")
    
def pprint_study(s):
    print(f"'{s.title}', {s.year} (ID: {s.id}, DOI: {s.doi})")
  
def pprint_finding(f):
	print(f"{f.variable_1_name} - ({f.statistic_type}: {f.statistic_value}) -> {f.variable_2_name}")

## 2. Create a service client and initialize endpoints

In [ ]:
%env YOUR_API_KEY=<your-api-key>

In [7]:
API_KEY = os.environ.get("YOUR_API_KEY")
config = api_client.Configuration(host = "https://api-dev.system.com/")

if not API_KEY:
		raise ValueError("API key is missing")

# config.api_key['x-api-key'] = API_KEY
system_client = api_client.ApiClient(header_name="X-API-KEY", header_value=API_KEY, configuration=config)

topics_api = api_client.api.topics_api.TopicsApi(system_client)
relationships_api = api_client.api.relationships_api.RelationshipsApi(system_client)
studies_api = api_client.api.studies_api.StudiesApi(system_client)
variables_api = api_client.api.variables_api.VariablesApi(system_client)
findings_api = api_client.api.statistical_findings_api.StatisticalFindingsApi(system_client)

## 3. Search for and retrieve a topic

In [8]:
def search_topic_by_name(topic_name: str):
  response = topics_api.get_topics(search=f"name:{topic_name}")
  return response.data
  

In [ ]:
try: 
	topics = search_topic_by_name(topic_name="osteoporosis")
	for t in topics:
		pprint_topic(t)
except Exception as e:
	print(e)

In [29]:
def search_topic_by_wikidata_id(wikidata_id: str):
  response = topics_api.get_topics(filter=f"wikidata_id:{wikidata_id}")
  return response.data

In [46]:
topics_2 = search_topic_by_wikidata_id(wikidata_id="Q55950294")
for t in topics_2:
	pprint_topic(t)

postmenopausal osteoporosis (ID: ZwLPJsQaKE, Wikidata ID: Q55950294)


In [35]:
def get_topic_by_id(topic_id: str):
  return topics_api.get_topic_by_id(topic_id)

In [47]:
topic_id = "YCCUBPRxoV" # Topic ID for 'osteoporosis'
topic = get_topic_by_id(topic_id=topic_id) 
pprint_topic(topic)

osteoporosis (ID: YCCUBPRxoV, Wikidata ID: Q165328)


## 4. Retrieve variables and determinants associated with a topic

In [50]:
def get_variables_by_topic_id(topic_id: str):
  response = variables_api.get_variables(filter=f"topic.id:{topic_id}", limit=15)
  return response.data

In [48]:
variable_id = get_variables_by_topic_id(topic_id=topic_id)
for v in variable_id:
	print(v.json())

In [54]:
def get_determinants_by_topic_id(topic_id: str):
  response = topics_api.get_topic_determinants(topic_id, filter="num_significant_findings:>1", sort="median_effect_size:desc")
  return response.data

In [ ]:
determinants = get_determinants_by_topic_id(topic_id=topic_id)
for d in determinants:
	pprint_determinant(d)

## Search for and retrieve a study

In [28]:
def get_study_by_doi(doi: str):
	response = studies_api.get_studies(filter=f"doi:{doi}")
	return response.data

In [29]:
study = get_study_by_doi(doi="10.1016/j.bone.2010.03.017")

Get study by DOI: 
           id                         doi  \
0  BwNHkghmPt  10.1016/j.bone.2010.03.017   

                                               title  \
0  Prevalence and risk factors of osteoporosis in...   

                                             authors journal publish_date  \
0  [{'full_name': 'Chan Soo Shin', 'orcid': '0000...    Bone   2010-08-01   

  population  cited_by retraction_reason  
0       None       117              None  



## 6. Retrieve statistical findings from relationships or studies

In [30]:
def get_findings_by_relationship_id(relationship_id: str):
  print("Get findings by relationship ID: ")
  fetched_findings = relationships_api.get_relationship_statistical_findings(relationship_id)
  findings_json = [json.loads(finding.json()) for finding in fetched_findings.data]
  df = pd.DataFrame(findings_json)
  df = df[_FINDING_COLUMNS]
  print(df)
  print("")
  return df['id'][1]

In [32]:
finding_from_relationship = get_findings_by_relationship_id("ELjr0Yjdsc|YCCUBPRxoV")

Get findings by relationship ID: 
           id                                         variable_1  \
0  GjA1LF3Lj9  {'id': '3V4swOcx1l', 'name': 'Postmenopausal R...   
1  E_BM4j2unD  {'id': '-ZCF1D2ouM', 'name': 'post-menopause s...   
2  ye22t7nhbh  {'id': 'Pl93A9-O_D', 'name': 'postmenopausal s...   
3  aGx8oVkn2o  {'id': 'VQVyc1mbWS', 'name': 'Postmenopausal w...   
4  LtBL-kaCCf  {'id': 'C5UOcrWu3g', 'name': 'postmenopausal w...   
5  B1OQ2yY7fx  {'id': 'VQVyc1mbWS', 'name': 'Postmenopausal w...   
6  AAdXio0yX6  {'id': 'eLsyYFQuow', 'name': 'Post-menopausal ...   
7  lyKucekQM5  {'id': 'cdraWaZ_On', 'name': 'postmenopausal f...   
8  nZRejgEftk  {'id': 'CKLH8oolgJ', 'name': 'Postmenopausal w...   
9  iYgCcdLnz2  {'id': '5IzHJFKc9Q', 'name': 'postmenopausal w...   

                   statistic_type  statistic_value  ci_upper  ci_lower  \
0  mean_difference_unstandardized            9.600       NaN       NaN   
1                      odds_ratio            7.320    17.600     3.05

In [36]:
def get_findings_by_study_id(study_id: str):
	print("Get findings by study ID: ")
	fetched_findings = studies_api.get_study_statistical_findings(study_id)
	findings_json = [json.loads(finding.json()) for finding in fetched_findings.data]
	df = pd.DataFrame(findings_json)
	df = df[_FINDING_COLUMNS]
	print(df)
	print("")
	return df['id'][0]

In [37]:
finding_from_study = get_findings_by_study_id(study_id)

Get findings by study ID: 
           id                                         variable_1  \
0  UgW-Nd8K5c  {'id': 'KWPYSf516x', 'name': 'having more than...   
1  C6TEWl_9DL  {'id': 'ikQSAGll0n', 'name': 'menarche after a...   
2  E_BM4j2unD  {'id': '-ZCF1D2ouM', 'name': 'post-menopause s...   
3  hH8_zLfcjv  {'id': 'hazlzJQfcm', 'name': 'last delivery af...   
4  VzaKeg1O7Q  {'id': 'ZSRCgqrjON', 'name': 'more than 17 yea...   
5  EZnBmNHRv6  {'id': 'd4EXjJcR2n', 'name': 'smoking> or =1 p...   
6  Z_pdKSy6EH  {'id': 'XatSLZcfba', 'name': 'past fracture hi...   
7  lx6EfFujo0  {'id': 'R7CR1MHpOA', 'name': 'calcium intake> ...   
8  p9OFTXZYY6  {'id': 'dIxvV7CyDq', 'name': 'monthly income a...   
9  1G5qAWF7ng   {'id': 'ds-IcQ6tDc', 'name': 'college graduate'}   

  statistic_type  statistic_value  ci_upper  ci_lower  p_value  
0     odds_ratio             1.42      1.89      1.07    0.001  
1     odds_ratio             1.46      1.87      1.14    0.001  
2     odds_ratio             